In [ ]:
# Do not modify this
%autosave 60 
import gdown
import pandas as pd

# Download the training data from drive
trainURL = "https://drive.google.com/uc?id=11WSDMvwLB5jn3Bjw5KT4LkaIdvtQCl38"
gdown.download(trainURL, "train.csv", quiet=True)

# Read in the training data to a pandas dataframe called train
train = pd.read_csv("train.csv")
train.head()

Autosaving every 60 seconds


,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
0,1,500.0,2/22/2011,Government Of Tanzania,1802,DWE,34.910035,-9.203799,Kwa Neckson Kiyao,Rufiji,Igominyi,Iringa,11,4,Njombe,Ikuka,80,True,GeoData Consultants Ltd,VWC,NaN,False,2008,swn 80,swn 80,handpump,vwc,user-group,pay monthly,monthly,soft,good,enough,enough,hand dtw,borehole,groundwater,hand pump,hand pump,1
1,2,0.0,3/14/2011,Mkinga Distric Coun,0,DWE,38.825484,-4.798793,Shule,Pangani,Shuleni Kiumbo,Tanga,4,8,Mkinga,Maramba,50,True,GeoData Consultants Ltd,VWC,Bwiti gravity water,True,1971,gravity,gravity,gravity,vwc,user-group,never pay,never pay,coloured,colored,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,0
2,3,0.0,7/31/2011,Hesawa,0,HESAWA,32.962557,-3.108364,Kwa Mwan,Lake Victoria,Ifumba B,Mwanza,19,7,Missungwi,Busongo,0,True,GeoData Consultants Ltd,VWC,NaN,False,0,afridev,afridev,handpump,vwc,user-group,pay annually,annually,soft,good,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump,0
3,4,0.0,8/2/2011,Adb,1188,DWE,33.067806,-2.097464,none,Lake Victoria,Mkwajuni,Mwanza,19,1,Ukerewe,Bukanda,500,True,GeoData Consultants Ltd,NaN,NaN,True,2009,india mark ii,india mark ii,handpump,vwc,user-group,never pay,never pay,soft,good,insufficient,insufficient,shallow well,shallow well,groundwater,hand pump,hand pump,1
4,5,500.0,3/4/2013,District Council,1304,District Council,37.658595,-3.695075,Store Mwambasa,Pangani,Kilaweni,Kilimanjaro,3,2,Mwanga,Kighare,250,True,GeoData Consultants Ltd,WUA,Ilamalamsuka water supply,False,1982,gravity,gravity,gravity,wua,user-group,never pay,never pay,soft,good,insufficient,insufficient,spring,spring,groundwater,communal standpipe,communal standpipe,1


In [ ]:
# Download file with features in the test set
testFeaturesURL = "https://drive.google.com/uc?id=1C64MAXqe4R4UjFBJcyK7hmKcNIVj3C8x"
gdown.download(testFeaturesURL, "test_features.csv", quiet=True)

# Read test set features into a pandas data frame
testFeatures = pd.read_csv("test_features.csv")
testFeatures.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,47521,6000.0,3/14/2011,Roman,1390,Roman,34.934429,-9.859748,none,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,Roman,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,47522,0.0,1/28/2013,Unicef,263,UNICEF,38.481475,-11.147901,Zahanati Ya Nanyumbu,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58,True,GeoData Consultants Ltd,VWC,NaN,True,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
2,47523,0.0,10/14/2012,World Vision,0,World vision,33.802775,-3.291305,Pamba,Internal,Nkilifa,Shinyanga,17,2,Maswa,Busilili,0,True,GeoData Consultants Ltd,WUG,NaN,False,0,nira/tanira,nira/tanira,handpump,wug,user-group,other,other,soft,good,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump
3,47524,0.0,3/15/2013,Lawatefuka Water Supply,1368,Lawatefuka water sup,37.086057,-3.181949,Kwa John Izack Mmari,Pangani,Omarini,Kilimanjaro,3,7,Siha,Siha Kaskazini,1,True,GeoData Consultants Ltd,Water Board,BL Bondeni,True,2009,gravity,gravity,gravity,water board,user-group,pay monthly,monthly,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
4,47525,0.0,10/20/2012,Biore,0,WEDECO,34.365589,-3.626754,Mwabasabi,Internal,Mwabasabi,Shinyanga,17,6,Meatu,Nkoma,0,True,GeoData Consultants Ltd,WUG,None,True,0,nira/tanira,nira/tanira,handpump,wug,user-group,never pay,never pay,soft,good,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump


In [ ]:
import numpy as np
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

In [ ]:
train1 = train.copy()
test1 = testFeatures.copy()
combine = [train1, test1]

train1['lga_group'] = train1['lga']
train1.loc[train1['lga'].str.contains('Urban') == True, 'lga_group'] = 'Urban'
train1.loc[train1['lga'].str.contains('Rural') == True, 'lga_group'] = 'Rural'
uncommonLga = train1[['lga_group']].value_counts()[35:].reset_index()['lga_group'].tolist()
uncommonWard = train1[['ward']].value_counts()[35:].reset_index()['ward'].tolist()

for d in combine:
  amount_tsh_dict = d[d['amount_tsh'] != 0].groupby('source_class')['amount_tsh'].median()
  d['amount_tsh_enc'] = d['amount_tsh']
  d['amount_tsh_enc'] = d['source_class'].apply(lambda x: amount_tsh_dict[x])
  d.loc[d['amount_tsh'] != 0, 'amount_tsh_enc'] = d['amount_tsh']
  d['amount_tsh_clip'] = np.clip( d['amount_tsh_enc'], np.percentile(d['amount_tsh_enc'], [0,99])[0], np.percentile(d['amount_tsh_enc'], [0,99])[1]) # winsorization
  
  
  # drop 'date_recorded'
  d['date_recorded_year'] = d['date_recorded'].str.split('/',expand = True)[2].astype(int)
  d['date_recorded_month'] = d['date_recorded'].str.split('/',expand = True)[0].astype(int)
  d['date_recorded_season'] = d['date_recorded_month']
  d['date_recorded_season'] = d['date_recorded_season'].replace([12,1,2], 1)
  d['date_recorded_season'] = d['date_recorded_season'].replace([3,4,5], 2)
  d['date_recorded_season'] = d['date_recorded_season'].replace([6,7,8], 3)
  d['date_recorded_season'] = d['date_recorded_season'].replace([9,10,11], 4)
  d['date_recorded_season'] = d['date_recorded_season'].astype('category')

  # drop gps_height
  d['gps_height_group'] = ''
  d.loc[d['gps_height'] < 80, 'gps_height_group'] = '<80'
  d.loc[(d['gps_height'] >= 80) & (d['gps_height'] < 850) , 'gps_height_group'] = '80-850'
  d.loc[d['gps_height'] >= 850 , 'gps_height_group'] = '>=850'
  d['gps_height_group'] = d['gps_height_group'].astype(str).astype('category')

  # # drop wpt_name
  # # drop latitude and longitude ?
  # # how to use district code, region code ? keep region
  # # how to categorize lga, ward ?

  d['population'] = np.clip( d['population'], np.percentile(d['population'], [1,99])[0], np.percentile(d['population'], [1,99])[1]) # winsorization
  
  d['public_meeting'] = d['public_meeting'].astype(str).astype('category')
  d['permit'] = d['permit'].astype(str).astype('category')

  # drop construction year
  d['construction_year_group'] = '0'
  d.loc[(d['construction_year'] >= 1960) & (d['construction_year'] < 1997), 'construction_year_group'] = '1960-1997'
  d.loc[(d['construction_year'] >= 1997), 'construction_year_group'] = '>=1997'
  d['construction_year_group'] = d['construction_year_group'].astype('category')

  # add a age
  d['age'] = -9
  d.loc[d['construction_year'] != 0, 'age'] = d.loc[d['construction_year'] != 0,'date_recorded_year'] - d.loc[d['construction_year'] != 0,'construction_year']
  d.loc[d['age'] < 0, 'age'] = np.mean(d.loc[d['age'] >= 0, 'age'])
  d['age'] = d['age'].astype(int)

  # drop extraction_type extraction_type_group, keep extraction_type_class
  d['extraction_type_class'] = d['extraction_type_class'].astype('category')

  # drop management_group, keep management
  d['management'] = d['management'].astype('category')

  # drop payment, keep payment_type
  d['payment_type'] = d['payment_type'].astype('category')

  # drop water_quality, keep quality_group
  d['quality_group'] = d['quality_group'].astype('category')

  # drop quantity_group, keep quantity
  d['quantity'] = d['quantity'].astype('category')

  # drop source_type, source_class, keep source
  d['source'] = d['source'].astype('category')

  # drop waterpoint_type_group, keep waterpoint_type
  d['waterpoint_type'] = d['waterpoint_type'].astype('category')

  # longitude
  longitude_dict = d[d['longitude']>10].groupby('region')['longitude'].median()
  d['longitude_encode'] = d['longitude']
  d['longitude_encode'] = d['region'].apply(lambda x: longitude_dict[x])
  d.loc[d['longitude'] > 10, 'longitude_encode'] = d['longitude']
  d['longitude_cat'] = d['longitude_encode']
  d.loc[d['longitude_encode'] < 34.1, 'longitude_cat'] = 0
  d.loc[(d['longitude_encode'] >= 34.1) & (d['longitude_encode'] <= 37.4), 'longitude_cat'] = 1
  d.loc[(d['longitude_encode'] > 37.4), 'longitude_cat'] = 2
  d['longitude_cat'] = d['longitude_cat'].astype(str).astype('category')

  # latitude
  latitude_dict = d[d['latitude'] <= -1].groupby('region')['latitude'].median()
  d['latitude_encode'] = d['latitude']
  d['latitude_encode'] = d['region'].apply(lambda x: longitude_dict[x])
  d.loc[d['longitude'] <= -1, 'latitude_encode'] = d['latitude']
  d['latitude_band'] = pd.qcut(d['latitude_encode'], 10)
  label = LabelEncoder()
  d['latitude_band'] = label.fit_transform(d['latitude_band'])
  d['latitude_band'] = d['latitude_band'].astype(str).astype('category')

  # # lga
  d['lga_group'] = d['lga']
  d.loc[d['lga'].str.contains('Urban') == True, 'lga_group'] = 'Urban'
  d.loc[d['lga'].str.contains('Rural') == True, 'lga_group'] = 'Rural'
  d['lga_clip'] = d['lga_group']
  d['lga_clip'] = d['lga_clip'].replace(uncommonLga, 'Others')
  d['lga_clip'] = d['lga_clip'].astype('category')

  # # ward
  d['ward_clip'] = d['ward']
  d['ward_clip'] = d['ward_clip'].replace(uncommonWard, 'Others')
  d['ward_clip'] = d['ward_clip'].astype('category')


for d in combine:
  print(d.shape)
  # print(d.columns)


(47520, 55)
(11880, 54)


In [ ]:
from sklearn.preprocessing import OneHotEncoder
display(train1.columns)
keep = ['amount_tsh_clip', 'date_recorded_year', 'date_recorded_season', 'gps_height_group',
        'basin', 'region', 'population', 'public_meeting', 'permit', 'construction_year_group',
        'extraction_type_class', 'management', 'payment_type', 'quality_group', 'quantity', 'source', 
        'waterpoint_type','longitude_cat', 'latitude_band', 'age','lga_clip','ward_clip']
contFeature = ['amount_tsh_clip', 'date_recorded_year', 'population', 'age']
catFeature = [i for i in keep if i not in contFeature]

# for d in combine:
train2 = train1[keep + ['status_group']].copy()
test2 = test1[keep ].copy()

def encoding(train, test):
  enc = OneHotEncoder().fit(train[catFeature])
  enc_cattrain = enc.transform(train[catFeature]).toarray()
  enc_cattrain_df = pd.DataFrame(enc_cattrain, columns = enc.get_feature_names(catFeature))
  enc_train = pd.concat([train[contFeature + ['status_group']], enc_cattrain_df], axis = 1)
  enc_train['status_group'] = enc_train['status_group'].astype(str).astype('category')

  enc_cattest = enc.transform(test[catFeature]).toarray()
  enc_cattest_df = pd.DataFrame(enc_cattest, columns = enc.get_feature_names(catFeature))
  enc_test = pd.concat([train[contFeature], enc_cattest_df], axis = 1)
  return enc_train, enc_test

def encoding2(data, is_train = True):
  enc = OneHotEncoder().fit(data[catFeature])
  enc_cattrain = enc.transform(data[catFeature]).toarray()
  enc_cattrain_df = pd.DataFrame(enc_cattrain, columns = enc.get_feature_names(catFeature))
  if is_train:
    enc_train = pd.concat([data[contFeature + ['status_group']], enc_cattrain_df], axis = 1)
    enc_train['status_group'] = enc_train['status_group'].astype(str).astype('category')
  else:
    enc_train = pd.concat([data[contFeature], enc_cattrain_df], axis = 1)
  return enc_train

train3 = encoding2(train2, is_train = True)
test3 = encoding2(test2, is_train = False)

combine3 = [train3, test3]

for d in combine3:
  print(d.shape)
test3.columns[test3.isna().sum() > 0]
test3.isna().sum()

Index(['id', 'amount_tsh', 'date_recorded', 'funder', 'gps_height',
       'installer', 'longitude', 'latitude', 'wpt_name', 'basin', 'subvillage',
       'region', 'region_code', 'district_code', 'lga', 'ward', 'population',
       'public_meeting', 'recorded_by', 'scheme_management', 'scheme_name',
       'permit', 'construction_year', 'extraction_type',
       'extraction_type_group', 'extraction_type_class', 'management',
       'management_group', 'payment', 'payment_type', 'water_quality',
       'quality_group', 'quantity', 'quantity_group', 'source', 'source_type',
       'source_class', 'waterpoint_type', 'waterpoint_type_group',
       'status_group', 'lga_group', 'amount_tsh_enc', 'amount_tsh_clip',
       'date_recorded_year', 'date_recorded_month', 'date_recorded_season',
       'gps_height_group', 'construction_year_group', 'age',
       'longitude_encode', 'longitude_cat', 'latitude_encode', 'latitude_band',
       'lga_clip', 'ward_clip'],
      dtype='object')

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


(47520, 190)
(11880, 196)


amount_tsh_clip            0
date_recorded_year         0
population                 0
age                        0
date_recorded_season_1     0
                          ..
ward_clip_Ukata            0
ward_clip_Usuka            0
ward_clip_Vikindu          0
ward_clip_Wanging'ombe     0
ward_clip_Zinga/Ikerege    0
Length: 196, dtype: int64

In [ ]:
train2.to_csv('train2_1.csv')
test2.to_csv('test2_1.csv')

In [ ]:

def baseline_fn(clf, X_train, X_test, y_train, y_test, name = ''):
    scores = cross_validate(clf, X_train, y_train, scoring = ['neg_log_loss','accuracy'], return_train_score= True, cv=2)
    print(f"Train log loss ({name}): {-scores['train_neg_log_loss'].mean():.5f}")
    print(f"Validation log loss ({name}): {-scores['test_neg_log_loss'].mean():.5f}")
    print(f"Train accuracy ({name}): {scores['train_accuracy'].mean():.5f}")
    print(f"Validation accuracy ({name}): {scores['test_accuracy'].mean():.5f}")

In [ ]:

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
np.random.seed(42)

X_train, X_test, y_train, y_test = train_test_split(train3.drop(columns = 'status_group'), train3['status_group'], test_size=0.2, random_state=42)
rfclf = RandomForestClassifier(random_state = 42, max_depth = 20, min_samples_leaf = 1, min_samples_split = 5)
rfclf.fit(X_train, y_train)
baseline_fn(rfclf, X_train, X_test, y_train, y_test, name = 'RF')

Train log loss (RF): 0.33084
Validation log loss (RF): 0.43264
Train accuracy (RF): 0.87045
Validation accuracy (RF): 0.79824


In [ ]:

from sklearn.model_selection import GridSearchCV
np.random.seed(42)

max_depth = [int(x) for x in np.linspace(1, 80, num = 5)]
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]

random_grid = {
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               }

rf_grid = GridSearchCV(rfclf, random_grid, refit = True, cv = 2, verbose = False)
rf_grid.fit(train3.drop(columns = 'status_group'), train3['status_group'])
display(rf_grid.best_params_)
baseline_fn(rf_grid, X_train, X_test, y_train, y_test, name = 'RF')


{'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 5}

Train log loss (RF): 0.30570
Validation log loss (RF): 0.42990
Train accuracy (RF): 0.88494
Validation accuracy (RF): 0.79974


In [ ]:
baseline_fn(CLF, X_train, X_test, y_train, y_test, name = 'RF')

Train log loss (RF): 0.30570
Validation log loss (RF): 0.42990
Train accuracy (RF): 0.88494
Validation accuracy (RF): 0.79974


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
CLF = RandomForestClassifier(random_state = 42, max_depth = 20, min_samples_leaf = 1, min_samples_split = 5)
CLF.fit(train3.drop(columns = 'status_group'), train3['status_group'])
submission = CLF.predict_proba(test3)

In [ ]:
CLF.predict(test3)

array(['0', '1', '0', ..., '0', '1', '0'], dtype=object)

In [ ]:
submission
submission[:,1]

array([0.02906115, 0.99962138, 0.13992129, ..., 0.41595736, 0.79302215,
       0.02829548])

In [ ]:
my_submission = pd.DataFrame({'id': testFeatures.id, 'predicted': submission[:,1]})
my_submission
# my_submission.to_csv("submission.csv", index = False)

,id,predicted
0,47521,0.029061
1,47522,0.999621
2,47523,0.139921
3,47524,0.023768
4,47525,0.168576
...,...,...
11875,59396,0.909928
11876,59397,0.004122
11877,59398,0.415957
11878,59399,0.793022
